# Model Initiation

In [14]:
#Libary
import pandas as pd

In [15]:
import ixmp

mp = ixmp.Platform()

In [16]:
import message_ix

scenario = message_ix.Scenario(
    mp, model='Indonesia Prototype', scenario='baseline', version='new'
)

# Model Structure

<img src='Coal Messageix Structure.drawio.png' width='1000'>

In [17]:
horizon = list(range(2005,2051,1))
scenario.add_horizon(year=horizon, firstmodelyear=2023)


In [18]:
country='Indonesia'
scenario.add_spatial_sets({'country':country})

In [19]:
scenario.add_set('commodity',['coal','electricity'])
scenario.add_set('level',['primary','secondary','final'])
scenario.add_set('technology',['coal_bal','coal_ppl','coal_t_d','electricity_grid'])
scenario.add_set('mode','standard')

## Demand Parameter

In [20]:

coal_d = [
    57.18083, 60.03523, 63.04894, 66.23524, 69.58086, 73.11234, 76.81642, 79.94961, 
    83.16247, 86.46825, 89.86697, 93.34535, 96.92994, 100.60747, 104.37792, 108.22804, 
    112.18436, 116.23362, 120.36253, 124.58438, 128.88589, 133.26705, 137.72788, 142.2418, 
    146.80884, 151.4157, 156.08894, 160.80201
]#converted to GWa

electricity_d = [
    21.00799087, 21.65068493, 22.32191781, 23.03310502, 23.10844749, 23.16552511, 
    23.34589041, 23.46232877, 23.7956621, 24.1826484, 24.54680365, 24.89155251, 
    25.28652968, 25.64611872, 26.15068493, 26.65410959, 27.21461187, 27.71803653, 
    28.32762557, 28.94406393, 29.51712329, 30.16324201, 30.80022831, 31.4303653, 
    32.12442922, 32.77739726, 33.42694064, 34.06849315
]#converted to GWa

model_horizon = list(range(2023,2051,1))

coal_demand = pd.DataFrame(
    {
        'node':country,
        'commodity':'coal',
        'level':'final',
        'year': model_horizon,
        'time':'year',
        'value':coal_d,
        'unit':'GWa'
    }
)
coal_demand['value'] = coal_demand['value'].round(decimals=2)

electricity_demand = pd.DataFrame(
    {
        'node':country,
        'commodity':'electricity',
        'level':'final',
        'year': model_horizon,
        'time':'year',
        'value':electricity_d,
        'unit':'GWa'
    }
)

electricity_demand['value'] = electricity_demand['value'].round(decimals=2)



In [21]:
scenario.add_par('demand',coal_demand)
scenario.add_par('demand',electricity_demand)

## Engineering Parameters

In [22]:
year_df = scenario.vintage_and_active_years()
vintage_years,act_years = year_df["year_vtg"],year_df["year_act"]

In [ ]:
#Coal Input

base_coal_input = {
    "node_loc":country,
    "year_vtg":vintage_years,
    "year_act":act_years,
    "mode":"standard",
    "node_origin":country,
    "commodity":"coal",
    "time":"year",
    "time_origin":"year"
}

coal_bal = pd.DataFrame(
    dict(
        technology='coal_bal',
        level='primary',
        value = 1,
        unit="-",
        **base_coal_input,
    )
)

coal_t_d =pd.DataFrame(
    dict(
        technology='coal_t_d',
        level='secondary',
        value=1,
        unit="-",
        **base_coal_input,
    )
)

coal_ppl = pd.DataFrame(
    dict(
        technology='coal_ppl',
        level='primary',
        value=1,
        unit='-',
        **base_coal_input,
    )
)

scenario.add_par("input",coal_bal)
scenario.add_par("input",coal_t_d)
scenario.add_par("input",coal_ppl)



In [39]:
from message_ix import make_df

#Electricity Input
base_electr_input = {
    "node_loc":country,
    "year_vtg":vintage_years,
    "year_act":act_years,
    "mode":"standard",
    "node_origin":country,
    "commodity":"electricity",
    "time":"year",
    "time_origin":"year"
}

electr_grid = make_df(
    base_electr_input,
    technology='electricity_grid',
    level='secondary',
    value=1,
    unit='-',
)

scenario.add_par("input",electr_grid)



In [41]:
scenario.idx_names('output')

['node_loc',
 'technology',
 'year_vtg',
 'year_act',
 'mode',
 'node_dest',
 'commodity',
 'level',
 'time',
 'time_dest']

In [42]:
#Output
base_output = {
'node_loc':country,
'year_vtg':vintage_years,
'year_act':act_years,
'mode':'standard',
'node_dest':country,
'time':'year',
'time_dest':'year',   
}

#coal_bal
coal_bal = make_df(
    base_output,
    technology ='coal_bal',
    commodity ='coal',
    level = 'secondary',
    value = 0.9, #Assumption to be changed (10% losses)
    unit = '-'
    
)

#coal_ppl
coal_ppl = make_df(
    base_output,
    technology ='coal_ppl',
    commodity ='electricity',
    level = 'secondary',
    value = 0.3, #Assumption to be changed
    unit = '-'
    
)

#coal_t_d
coal_t_d = make_df(
    base_output,
    technology = 'coal_t_d',
    commodity = 'coal',
    level='final',
    value=0.9, #Assumption to be changed
    unit = '-'
)

#electricity grid
electr_grid = make_df(
    base_output,
    technology ='electricity_grid',
    commodity ='electricity',
    level = 'final',
    value = 0.9, #Assumption to be changed
    unit = '-'
)

scenario.add_par("output",coal_bal)
scenario.add_par("output",coal_ppl)
scenario.add_par("output",coal_t_d)
scenario.add_par("output",electr_grid)



In [43]:
scenario.par('output')

,node_loc,technology,year_vtg,year_act,mode,node_dest,commodity,level,time,time_dest,value,unit
0,Indonesia,coal_bal,2005,2023,standard,Indonesia,coal,secondary,year,year,0.9,-
1,Indonesia,coal_bal,2005,2024,standard,Indonesia,coal,secondary,year,year,0.9,-
2,Indonesia,coal_bal,2005,2025,standard,Indonesia,coal,secondary,year,year,0.9,-
3,Indonesia,coal_bal,2005,2026,standard,Indonesia,coal,secondary,year,year,0.9,-
4,Indonesia,coal_bal,2005,2027,standard,Indonesia,coal,secondary,year,year,0.9,-
...,...,...,...,...,...,...,...,...,...,...,...,...
3635,Indonesia,electricity_grid,2048,2049,standard,Indonesia,electricity,final,year,year,0.9,-
3636,Indonesia,electricity_grid,2048,2050,standard,Indonesia,electricity,final,year,year,0.9,-
3637,Indonesia,electricity_grid,2049,2049,standard,Indonesia,electricity,final,year,year,0.9,-
3638,Indonesia,electricity_grid,2049,2050,standard,Indonesia,electricity,final,year,year,0.9,-


In [44]:
scenario.par('input')

,node_loc,technology,year_vtg,year_act,mode,node_origin,commodity,level,time,time_origin,value,unit
0,Indonesia,coal_bal,2005,2023,standard,Indonesia,coal,primary,year,year,1.0,-
1,Indonesia,coal_bal,2005,2024,standard,Indonesia,coal,primary,year,year,1.0,-
2,Indonesia,coal_bal,2005,2025,standard,Indonesia,coal,primary,year,year,1.0,-
3,Indonesia,coal_bal,2005,2026,standard,Indonesia,coal,primary,year,year,1.0,-
4,Indonesia,coal_bal,2005,2027,standard,Indonesia,coal,primary,year,year,1.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...
3635,Indonesia,electricity_grid,2048,2049,standard,Indonesia,electricity,secondary,year,year,1.0,-
3636,Indonesia,electricity_grid,2048,2050,standard,Indonesia,electricity,secondary,year,year,1.0,-
3637,Indonesia,electricity_grid,2049,2049,standard,Indonesia,electricity,secondary,year,year,1.0,-
3638,Indonesia,electricity_grid,2049,2050,standard,Indonesia,electricity,secondary,year,year,1.0,-


In [ ]:
mp.close_db()